# Importamos las dependencias necesarias

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

# Configuración de dispositivo (GPU si está disponible)


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Transformaciones para normalizar los datos

In [3]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Aumentación de datos
    transforms.RandomCrop(32, padding=4),  # Aumentación de datos
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))  # Normalización
])

# Cargar CIFAR-10

In [4]:
batch_size = 128
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)


100.0%


# Cargar el modelo ResNet-18

In [6]:
# Cargar modelo predefinido de torchvision
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)  # pretrained=True si quieres usar pesos de ImageNet

# Cambiar la capa final para CIFAR-10 (10 clases en lugar de 1000 de ImageNet)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

# Mover a GPU si está disponible
model = model.to(device)

# Definir la función de pérdida y el optimizador

In [7]:
criterion = nn.CrossEntropyLoss()  # Función de pérdida para clasificación
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimizador Adam

# Entrenamiento del modelo

In [8]:
num_epochs = 10  # Número de épocas

for epoch in range(num_epochs):
    model.train()  # Modo entrenamiento
    running_loss = 0.0

    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  # Resetear gradientes
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Calcular pérdida
        loss.backward()  # Backpropagation
        optimizer.step()  # Actualizar pesos

        running_loss += loss.item()

    print(f"Época {epoch+1}/{num_epochs}, Pérdida: {running_loss/len(trainloader):.4f}")

Época 1/10, Pérdida: 1.0377
Época 2/10, Pérdida: 0.7345
Época 3/10, Pérdida: 0.6431
Época 4/10, Pérdida: 0.5912
Época 5/10, Pérdida: 0.5507
Época 6/10, Pérdida: 0.5148
Época 7/10, Pérdida: 0.4988
Época 8/10, Pérdida: 0.4682
Época 9/10, Pérdida: 0.4503
Época 10/10, Pérdida: 0.4317


# Evaluación del modelo en datos de prueba

In [9]:
model.eval()  # Modo evaluación
correct = 0
total = 0

with torch.no_grad():  # Desactivar cálculo de gradientes
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Obtener predicción
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Precisión en el conjunto de prueba: {accuracy:.2f}%")

Precisión en el conjunto de prueba: 81.54%
